## Inicialização


In [1]:
import pandas as pd

df = pd.read_parquet("./data/questoes_unificadas.parquet")
df["texto_original"] = df["enunciado"].fillna("") + " " + df["alternativas"].fillna("")
df.dropna(subset=["enunciado"], inplace=True)

## Limpar parte textual


- Aplica máscaras sobre números, símbolos pertinentes e URLs

- Remove marcadores de alternativas

- Remove pontuação

- Cria uma coluna mantendo acentuação e uma removendo

- Normaliza espaços


In [2]:
import re
import unicodedata

RE_URL = re.compile(r"https?://\S+|www\.\S+")
RE_OPTIONS_INLINE = re.compile(r"\s*[ABCDE]\s*[\)\.\-]\s*")
RE_MULTI_SPACES = re.compile(r"\s+")

special_map = {
    "Δ": "<delta>",
    "°": "<graus>",
    "º": "<graus>",
    "√": "<raiz>",
    "π": "<pi>",
    "Ω": "<ohm>",
    "λ": "<lambda>",
    "θ": "<theta>",
    "μ": "<mu>",
    "Σ": "<soma>",
    "₀": "0",
    "₁": "1",
    "₂": "2",
    "₃": "3",
    "₄": "4",
    "₅": "5",
    "₆": "6",
    "₇": "7",
    "₈": "8",
    "₉": "9",
    "⁰": "0",
    "¹": "1",
    "²": "2",
    "³": "3",
    "⁴": "4",
    "⁵": "5",
    "⁶": "6",
    "⁷": "7",
    "⁸": "8",
    "⁹": "9",
}


def strip_accents(s: str) -> str:
    return "".join(
        c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c) != "Mn"
    )


def normalize_hyphens_quotes(s: str) -> str:
    s = s.replace("–", "-").replace("—", "-")
    s = s.replace("“", '"').replace("”", '"').replace("’", "'").replace("´", "'")
    return s


def clean_question(text: str, mask_numbers: bool = True):
    if not text or not isinstance(text, str):
        return ""

    # unicode + normalizações simples + aplicação de máscara para símbolos importantes
    t = text
    t = t.translate(str.maketrans(special_map))

    t = t.replace("ı", "i")
    t = unicodedata.normalize("NFKC", t)
    t = normalize_hyphens_quotes(t)

    t = t.translate(str.maketrans(special_map))

    t = t.casefold()

    # máscaras para urls
    t = RE_URL.sub(" <url> ", t)

    # remove marcadores de alternativas
    t = RE_OPTIONS_INLINE.sub(" ", t)

    # remove marcadores de texto
    t = re.sub(r"\btexto\s+(?:i+|\d+)\b", "", t)

    # números e unidades
    if mask_numbers:
        # substitui números por token (mantém % e °celsius próximos)
        t = re.sub(r"(?<![a-zA-Z])\d+[.,]?\d*(?![a-zA-Z])", " <NUM> ", t)

    # remove pontuação (mantém %, /, <, >  e -)
    t = re.sub(r"[^\w\s/%<>-]", " ", t)  # remove sinais exceto alguns

    # colapsa espaços
    t = RE_MULTI_SPACES.sub(" ", t).strip()

    return t


# Limpa
df["texto_clean"] = df["texto_original"].apply(clean_question)

# Dropa duplicatas
df.drop_duplicates(subset=["topico", "texto_clean"], keep="first", inplace=True)

In [3]:
import re
import unicodedata

RE_URL = re.compile(r"https?://\S+|www\.\S+")
RE_OPTIONS_INLINE = re.compile(r"\s*[ABCDE]\s*[\)\.\-]\s*")
RE_MULTI_SPACES = re.compile(r"\s+")

special_map = {
    "Δ": "<delta>",
    "°": "<graus>",
    "º": "<graus>",
    "√": "<raiz>",
    "π": "<pi>",
    "Ω": "<ohm>",
    "λ": "<lambda>",
    "θ": "<theta>",
    "μ": "<mu>",
    "Σ": "<soma>",
}


def normalize_hyphens_quotes(s: str) -> str:
    s = s.replace("–", "-").replace("—", "-")
    s = s.replace("“", '"').replace("”", '"').replace("’", "'").replace("´", "'")
    return s


def clean_question(text: str, mask_numbers: bool = True):
    if not text or not isinstance(text, str):
        return ""

    # unicode + normalizações simples + aplicação de máscara para símbolos importantes
    t = text
    t = t.translate(str.maketrans(special_map))

    t = t.replace("ı", "i")
    t = unicodedata.normalize("NFKC", t)
    t = normalize_hyphens_quotes(t)

    t = t.translate(str.maketrans(special_map))

    # máscaras para urls
    t = RE_URL.sub(" <url> ", t)

    # remove marcadores de alternativas
    t = RE_OPTIONS_INLINE.sub(" ", t)

    # remove marcadores de texto
    t = re.sub(r"\btexto\s+(?:i+|\d+)\b", "", t)

    # números e unidades
    if mask_numbers:
        # substitui números por token (mantém % e °celsius próximos)
        t = re.sub(r"(?<![a-zA-Z])\d+[.,]?\d*(?![a-zA-Z])", " <num> ", t)

    # remove pontuação (mantém %, /, <, >  e -)
    t = re.sub(r"[^\w\s/%<>-]", " ", t)  # remove sinais exceto alguns

    # colapsa espaços
    t = RE_MULTI_SPACES.sub(" ", t).strip()

    # remove marcadores de texto (novamente, caso tenha sobrado algo)
    t = re.sub(r"\btexto\s+(?:i+|\d+|<num>)", "", t)

    return t


# Limpa
df["texto_clean_cased"] = df["texto_original"].apply(clean_question)

In [4]:
import spacy


# ------- STOPWORDS -------
spacy.cli.download("pt_core_news_sm")
nlp = spacy.load("pt_core_news_sm")

# stopwords padrão
stopwords_spacy = set(nlp.Defaults.stop_words)

# negações que queremos manter
manter = {
    "mínimo",
    "minimo",
    "maximo",
    "máximo",
    "área",
    "area",
    "possível",
    "possivel",
    "ponto",
    "pontos",
    "não",
    "nao",
    "nem",
    "nunca",
    "jamais",
    "sem",
}

# remove essas negações da lista de stopwords
stopwords_custom = stopwords_spacy - manter


def batch_remove_stopwords(texts, stopwords_custom, lemmatize=False, batch_size=512):
    result = []
    with nlp.select_pipes(
        disable=["parser", "ner"]
    ):  # desabilita partes que não são necessárias para esse processo
        for doc in nlp.pipe(
            texts, batch_size=batch_size, disable=["parser", "ner"], n_process=8
        ):  # processa em batches
            tokens = [
                tok.lemma_ if lemmatize else tok.text
                for tok in doc
                if tok.text not in stopwords_custom
            ]
            result.append(" ".join(tokens))
    return result


remap = {
    "< delta >": "<delta>",
    "< graus >": "<graus>",
    "< graus >": "<graus>",
    "< raiz >": "<raiz>",
    "< pi >": "<pi>",
    "< ohm >": "<ohm>",
    "< lambda >": "<lambda>",
    "< theta >": "<theta>",
    "< mu >": "<mu>",
    "< soma >": "<soma>",
    "< NUM >": "<num>",
}


# Função para consertar as máscaras no texto
def conserta_mascaras(t: str) -> str:
    for old, new in remap.items():
        t = t.replace(old, new)
    return t


# Remove stopwords
df["texto_sw"] = batch_remove_stopwords(df["texto_clean"].to_list(), stopwords_custom)
df["texto_sw"] = df["texto_sw"].apply(conserta_mascaras)

df["texto_lem"] = batch_remove_stopwords(
    df["texto_clean"].to_list(), stopwords_custom, lemmatize=True
)
df["texto_lem"] = df["texto_lem"].apply(conserta_mascaras)

df["texto_clean"] = df["texto_clean"].apply(lambda x: x.replace("<NUM>", "<num>"))

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 94.5 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


### Stemming


In [5]:
import nltk
from nltk.stem import RSLPStemmer  # Para português

# Baixe o dicionário do nltk
nltk.download("rslp")

# Escolha o stemmer adequado
stemmer = RSLPStemmer()  # Para português


def stemmer_function(text):
    return " ".join(stemmer.stem(word) for word in text.split())


# Função para aplicar o stemming
df["texto_stem"] = df["texto_sw"].apply(stemmer_function)

[nltk_data] Downloading package rslp to /home/murilob/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


In [6]:
df["texto_clean"] = df["texto_clean"].astype(str).apply(lambda x: x.lower())
df["texto_sw"] = df["texto_sw"].astype(str).apply(lambda x: x.lower())
df["texto_lem"] = df["texto_lem"].astype(str).apply(lambda x: x.lower())
df["texto_stem"] = df["texto_stem"].astype(str).apply(lambda x: x.lower())

df["texto_clean"] = (
    df["texto_clean"]
    .astype(str)
    .apply(lambda x: re.sub(r"\btexto\s+(?:i+|\d+|<num>)", "", x))
)
df["texto_sw"] = (
    df["texto_sw"]
    .astype(str)
    .apply(lambda x: re.sub(r"\btexto\s+(?:i+|\d+|<num>)", "", x))
)
df["texto_lem"] = (
    df["texto_lem"]
    .astype(str)
    .apply(lambda x: re.sub(r"\btexto\s+(?:i+|\d+|<num>)", "", x))
)
df["texto_stem"] = (
    df["texto_stem"]
    .astype(str)
    .apply(lambda x: re.sub(r"\btexto\s+(?:i+|\d+|<num>)", "", x))
)

In [7]:
# mantém apenas as colunas desejadas
df_final = df[
    [
        "materia",
        "topico",
        "texto_clean",
        "texto_clean_cased",
        "texto_sw",
        "texto_lem",
        "texto_stem",
    ]
]
df_final = df_final.copy()
df_final.dropna(subset=["texto_clean"], inplace=True)
df_final["id"] = range(1, len(df_final) + 1)

## Salva


In [8]:
df_final.to_parquet("./data/questoes_tratadas.parquet", index=False)